##### Copyright 2020 The TensorFlow Hub Authors.


In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classify text with BERT

This fine-tuned BERT model is trained to classify tweets into 3 labels, 0 = "hate speech", 1 = "offensive language", 2 = "neither". 

Trained on https://github.com/t-davidson/hate-speech-and-offensive-language

testing dataset from https://github.com/hate-alert/HateXplain

## Setup


In [ ]:
# A dependency of the preprocessing for BERT inputs
#!pip install -q -U "tensorflow-text==2.8.*"
#!pip install -q tf-models-official==2.7.0

## Loading in packages and initilizing constants

In [2]:
import os
import shutil

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
 
tf.get_logger().setLevel('ERROR')
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42
test_train_ratio = 0.1

## Loading in datasets

hate_data holds the train and validaiton dataset from the hatespeech dataset

train_ds, val_ds, and test_ds are batched and shuffled tf.data.Dataset 

In [8]:
dataset = load_dataset("hatexplain", split="validation")

class_names = ["hate speech", "normal", "offensive"]

data_tokens = tokenizer(dataset['post_tokens'], padding=True,is_split_into_words=True, return_tensors="tf")


input_tensor = []
attention_mask_tensor = []
for i in range(0,len(data_tokens.input_ids), batch_size):
    input_tensor += [data_tokens.input_ids[i:i+batch_size]]
    attention_mask_tensor += [data_tokens.attention_mask[i:i+batch_size]]

## Loading the BERT model


In [13]:
ataset_name = 'hate_speech'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

reloaded_model = tf.saved_model.load(saved_model_path)